###  First-Class Functions and Lambdas

In [ ]:
# 1. Pass Function as Argument
def apply(func, value):
    return func(value)

def square(x):
    return x * x

print(apply(square, 5))

# 2. Return Function from Function
def make_doubler():
    def doubler(x):
        return x * 2
    return doubler

double = make_doubler()
print(double(4))

# 3. Store Functions in a List
funcs = [abs, str, hex]
for f in funcs:
    print(f(-42))

# 4. Use map with Lambda
numbers = [1, 2, 3]
squared = list(map(lambda x: x**2, numbers))
print(squared)

# 5. Use filter with Lambda
evens = list(filter(lambda x: x % 2 == 0, range(10)))
print(evens)

# 6. Sort with Lambda Key
pairs = [(1, "b"), (2, "a")]
sorted_pairs = sorted(pairs, key=lambda x: x[1])
print(sorted_pairs)

# 7. Inline Function Composition
def compose(f, g):
    return lambda x: f(g(x))

add_one = lambda x: x + 1
square = lambda x: x * x
add_one_then_square = compose(square, add_one)
print(add_one_then_square(2))

# 8. Closure with Lambda
def make_multiplier(n):
    return lambda x: x * n

triple = make_multiplier(3)
print(triple(5))

### Decorators

In [ ]:
import time
import functools

def simple_logger(func):
    def wrapper(*args, **kwargs):
        print("Function started")
        result = func(*args, **kwargs)
        print("Function ended")
        return result
    return wrapper

def prefix_printer(prefix):
    def decorator(func):
        def wrapper(*args, **kwargs):
            print(f"{prefix} {func.__name__}")
            return func(*args, **kwargs)
        return wrapper
    return decorator

def timer(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f"Time taken: {end - start:.4f} seconds")
        return result
    return wrapper

def memoize(func):
    cache = {}
    def wrapper(*args):
        if args in cache:
            return cache[args]
        result = func(*args)
        cache[args] = result
        return result
    return wrapper

def debug_info(func):
    def wrapper(*args, **kwargs):
        print(f"Calling {func.__name__} with args: {args}, kwargs: {kwargs}")
        result = func(*args, **kwargs)
        print(f"{func.__name__} returned: {result}")
        return result
    return wrapper

def role_required(required_role):
    def decorator(func):
        def wrapper(user, *args, **kwargs):
            if user.get('role') == required_role:
                return func(user, *args, **kwargs)
            else:
                raise PermissionError("Insufficient permissions")
        return wrapper
    return decorator

def retry(max_attempts):
    def decorator(func):
        def wrapper(*args, **kwargs):
            attempts = 0
            while attempts < max_attempts:
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    attempts += 1
                    if attempts == max_attempts:
                        raise
                    print(f"Attempt {attempts} failed, retrying...")
        return wrapper
    return decorator

def custom_logger(message):
    def decorator(func):
        def wrapper(*args, **kwargs):
            print(f"Start: {message}")
            result = func(*args, **kwargs)
            print(f"End: {message}")
            return result
        return wrapper
    return decorator

@simple_logger
def greet(name):
    print(f"Hello, {name}!")

@prefix_printer("LOG:")
def say_hello():
    print("Hello World")

@timer
def long_running_task():
    time.sleep(1)

@memoize
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-1) + fibonacci(n-2)

@debug_info
def add(a, b):
    return a + b

@role_required('admin')
def admin_action(user):
    print(f"Admin action performed by {user['name']}")

@retry(3)
def unreliable_function():
    import random
    if random.random() < 0.7:
        raise ValueError("Random failure")
    return "Success"

@custom_logger("Processing data")
def process_data():
    print("Processing...")
    time.sleep(0.5)

### functools Utilities

In [ ]:
from functools import partial, lru_cache, wraps, reduce

# 1. partial Function
binary_parse = partial(int, base=2)
print(binary_parse('1010'))  # 10

# 2. lru_cache Memoization
@lru_cache(maxsize=None)
def fib(n):
    return n if n < 2 else fib(n-1) + fib(n-2)

# 3. Function Metadata with wraps
def my_decorator(f):
    @wraps(f)
    def wrapper(*args, **kwargs):
        return f(*args, **kwargs)
    return wrapper

@my_decorator
def example():
    """Example docstring"""
    pass

print(example.__name__)  # example
print(example.__doc__)   # Example docstring

# 4. Chained Partials
print_info = partial(print, end="!\n")
print_hello = partial(print_info, "Hello")
print_hello()  # Hello!

# 5. Uncached Recursive Function
def uncached_fib(n):
    return n if n < 2 else uncached_fib(n-1) + uncached_fib(n-2)

# 6. reduce with Lambda
factorial = lambda n: reduce(lambda x, y: x*y, range(1, n+1), 1)
print(factorial(5))  # 120

# 7. Default Dict Generator
from collections import defaultdict
nested_dict = partial(defaultdict, lambda: defaultdict(dict))
data = nested_dict()
data['a']['b']['c'] = 1
print(data['a']['b']['c'])  # 1

# 8. Log Decorator with wraps
def log_decorator(f):
    @wraps(f)
    def wrapper(*args, **kwargs):
        print(f"Calling {f.__name__}")
        result = f(*args, **kwargs)
        print(f"Finished {f.__name__}")
        return result
    return wrapper

@log_decorator
def add(a, b):
    return a + b

print(add(2, 3))